In [1]:
import pandas as pd
import numpy as np

In [2]:
output = pd.read_csv('myCS/output/test2.c.txt',sep='\t',names=['type','line','level','code'])

linesList = output.line.value_counts().index.sort_values()[1:]

### Prepare list

In [3]:
ast = []
flag_open = False
flag_close = False
for linenum in linesList:
    statement = []
    line = output[output.line==linenum]
    leaf = line[line.type=='LEAF_NODE'].code.values

    if linenum == linesList[0]:
        statement.append('FuncDef')
        statement.append('Decl')
        statement.append('FuncDecl')
        
    if '{' in leaf:
        flag_open = True

    filtered_symbol = filter(lambda item: '{' not in item , leaf)
    filtered_symbol = filter(lambda item: '}' not in item , filtered_symbol)
    filtered_symbol = filter(lambda item: ';' not in item , filtered_symbol)
    filtered_symbol = filter(lambda item: '(' not in item , filtered_symbol)
    filtered_symbol = filter(lambda item: ')' not in item , filtered_symbol)
    filtered_symbol = filter(lambda item: ',' not in item , filtered_symbol)
    
    for item in filtered_symbol:
       statement.append(item)
    
    
    if len(statement)!=0:
        vardec = np.where(pd.Series(statement)=='=')[0]
        if len(vardec)>1:
            vardec_list = []
            for i in range(len(vardec)):
                vardec_list.append(statement[(0 if i==0 else vardec[i-1]+2):vardec[i]+2])
                
            for var_item in vardec_list:
                ast.append(pd.Series(var_item).unique().tolist())
        else:
            ast.append(statement)
    
    if flag_open == True:
        ast.append(['Compound'])
        flag_open = False
    
    if '}' in leaf:
        ast.append(['End'])


In [4]:
ast

[['FuncDef', 'Decl', 'FuncDecl', 'int', 'main'],
 ['Compound'],
 ['int', 'x', '=', '2'],
 ['int', 'y', '=', '3'],
 ['int', 'z', '=', '4'],
 ['x', '=', 'x', '+', 'x'],
 ['printf', '%d', 'x'],
 ['return', '0'],
 ['End']]